## Evaluating Regression Models

### What
- Since models are approximations of reality, we're measuring how closely our linear models approximate reality.
- With evaluating regression models, we'll calculate this closeness by measuring the error, which is the distance between the regression line (prediction) and actual data points.

### Why

Techniques in this lesson answer the following questions:
- How do we know if our model is good enough?
- How do we evaluate our model?
- Is our model valuable?
- Are our features valuable?

### How - what is our approach?
Prerequisites
- Establish a baseline (often the average of the target)
- Build a super simple model (or isolate the model we're evaluating)
- Calculate baseline and model residuals

Evaluate the model
- Calculate SSE, MSE, and RMSE (Root Mean Squared Error)
- Determine if the model is significant
- Determine if the features are significant

### When do we evaluate a model?
- As soon as we've got a super basic model, it's important to know if it beats baseline.
- If so, we'll keep on modeling, making new models, tuning, etc...
- If not, maybe a different model or different feature(s) will help.